#  Mô hình dự đoán cảm xúc của bài đánh giá phim
## Sử dụng PyTorch và SageMaker


## Kế hoạch

1. Tải dữ liệu.
2. Chuẩn bị và xử lý dữ liệu.
3. Tải dữ liệu lên S3.
4. Build và train PyTorch model.
5. Test model đã train.
6. Triển khai model đã train.
7. Sử dụng model đã triển khai.


In [3]:
# Make sure that we use SageMaker 1.x
!pip install sagemaker==1.72.0

  Using cached sagemaker-1.72.0-py2.py3-none-any.whl
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.60.0
    Uninstalling sagemaker-2.60.0:
      Successfully uninstalled sagemaker-2.60.0
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


## Bước 1: Tải  dữ liệu

Sử dụng tập dữ liệu IMDb [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/). Đây là tập dữ liệu để phân loại cảm xúc (nhị phân). Trong tập dữ liệu này có 25.000 bài đánh giá phim để training và 25.000 để test. Cũng có thêm dữ liệu chưa được gắn label để sử dụng. Ngoài ra còn có văn bản thô và tập hợp định dạng từ đã được xử lý.

In [4]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

mkdir: cannot create directory ‘../data’: File exists
--2021-11-20 14:15:50--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  25.7MB/s    in 3.1s    

2021-11-20 14:15:53 (25.7 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



# Bước 2: Chuẩn bị và xử lý dữ liệu

## Để bắt đầu, chúng ta sẽ đọc từng bài đánh giá và kết hợp chúng thành một cấu trúc đầu vào duy nhất. Sau đó, chúng tôi sẽ chia tập dữ liệu thành tập training và tập test.

In [5]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [6]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


## Bây giờ sẽ kết hợp và trộn các đánh giá tích cực và tiêu cực với nhau.

In [7]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [8]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


## Kiểm tra một vài dữ liệu

In [9]:
print(train_X[100])
print(train_y[100])

This movie was terrific and even with a less than convincing ending, it's still well worth seeing. The film begins as Claudette Colbert is about to marry Robert Ryan. When the minister asks if anyone has any objections, a guy jumps up and announces that Colbert CAN'T get married because she already is married!! Colbert insists this isn't true, but when they investigate they find that the Justice of the Peace and many others remember her wedding and there is even a signed wedding license! Slowly, it becomes apparent that Claudette's mind is slipping and people around her seriously doubt her sanity. Then, when the supposed first husband is murdered, all evidence and suspicion falls on Colbert.<br /><br />The film is an exciting mystery suspense film, as what I have so far described is only the first half of the movie. What follows is amazingly intelligent and captivating. Unfortunately, the conclusion, though, is a bit of a let-down, as the guiding force behind all this turns out to come

## Tinh chỉnh dữ liệu cho phù hợp (xóa các thẻ HTML, chỉnh thành chữ thường). Rồi lưu kết quả vào bộ nhớ cache.

In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

Phương thức `review_to_words` sử dụng `BeautifulSoup` để xóa bất kỳ thẻ html và sử dụng gói `nltk` để mã hóa các bài đánh giá. Để kiểm tra, hãy thử áp dụng `review_to_words` cho một trong các bài đánh giá trong bộ training.

In [11]:
print(' '.join(review_to_words(train_X[100])))

movi terrif even less convinc end still well worth see film begin claudett colbert marri robert ryan minist ask anyon object guy jump announc colbert get marri alreadi marri colbert insist true investig find justic peac mani other rememb wed even sign wed licens slowli becom appar claudett mind slip peopl around serious doubt saniti suppos first husband murder evid suspicion fall colbert film excit mysteri suspens film far describ first half movi follow amazingli intellig captiv unfortun conclus though bit let guid forc behind turn come right left field baffl sinc unexpect imposs guess base inform given viewer howev spite film good even excus limp end particular robert ryan great job knuckl bust fianc though apart perform also excel


Sử dụng `review_to_words` cho từng bài đánh giá trong tập train và tập test. Ngoài ra, nó lưu kết quả vào bộ nhớ cache.

In [12]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [13]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Read preprocessed data from cache file: preprocessed_data.pkl


## Chuyển đổi dữ liệu

Chúng ta tạo 1 tập từ điển bằng cách ánh xạ các từ xuất hiện trong các bài đánh giá với các số nguyên (số lần xuất hiện). Ở đây chúng ta cố định kích thước từ là 5000 từ (trong đó có 2 từ là các khoảng trắng và các từ xuất hiện không thường xuyên tức các từ có tần suất xuất hiện thuộc top 4999 trở về sau được gom thành 1 nhóm). Và chúng ta gắn cho 2 loại từ đó là `0` đối với các khoảng trắng và `1` đối với các từ có tần suất ít. 

In [14]:
import numpy as np

def build_dict(data, vocab_size = 5000):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    
    # Determine how often each word appears in `data`. Note that `data` is a list of sentences and that a
    # sentence is a list of words.
    
    word_count = {} # A dict storing the words that appear in the reviews along with how often they occur
    for d in data:
        for w in d:
            word_count[w] = word_count.get(w, 0) + 1
    # Sort the words found in `data` so that sorted_words[0] is the most frequently appearing word and
    # sorted_words[-1] is the least frequently appearing word.
    
    # sorted_words = list(dict(sorted(word_count.items(), key=lambda item: item[1], reverse=True)).items())
    sorted_words = sorted(word_count, key=word_count.get, reverse=True)
    
    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [15]:
word_dict = build_dict(train_X)

In [16]:
# Use this space to determine the five most frequently appearing words in the training set.
for word in list(word_dict.keys())[:5]:
    print(word)

movi
film
one
like
time


### Lưu `word_dict`

In [17]:
data_dir = '../data/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [18]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

### Chuyển đổi các bài đánh giá

Đối với mỗi bình luận chúng ta lấy kích thước cố định là `500` từ. 

In [19]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [20]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

Kiểm tra một vài bài đánh giá sau khi chuyển đổi.

In [21]:
#  Use this cell to examine one of the processed reviews to make sure everything is working as intended.
print(train_X[0])

[ 580 1793 1658  397  496   41   92  199   42  927  397  657  841    1
 2004    1    1   53   32  143 3756    1  802    1    7  797   95  387
  253 1658 1024  277   10  909    1  841 1780  909  763 2907 1769  133
  254    1    1  325    1  547    1    1   81   33  227    1  802 1733
   82  260 4472   39  842 2481   73 1747    4  174    3  415  357  310
   43    1   81    1  310    2  218    4   19  411  252 1658  252 1416
    3  411  734 3585  900    9  696  683    4    1 1522    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

## Bước 3: Tải dữ liệu lên S3

Tải tập dữ liệu training lên S3 để mã training có thể truy cập vào nó. Hiện tại, lưu nó cục bộ và sẽ tải lên S3 sau này.

### Lưu bộ dữ liệu đào tạo đã xử lý cục bộ

Mỗi hàng của tập dữ liệu có dạng `label`,`length`, `review[500]` trong đó `review[500]` là một chuỗi các số nguyên `500` đại diện cho các từ trong bài đánh giá.

In [22]:
import pandas as pd
    
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

### Upload dữ liệu training

Tiếp theo, tải dữ liệu đào tạo lên AWS S3 mặc định của SageMaker để có thể cung cấp quyền truy cập vào nó trong khi đào tạo mô hình.

In [23]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

role = sagemaker.get_execution_role()

In [24]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

**NOTE:** Ô ở trên tải lên toàn bộ nội dung của thư mục data, bao gồm tệp `word_dict.pkl`.

## Bước 4: Build và Train PyTorch Model

In [25]:
!pygmentize train/model.py

import torch.nn as nn

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig

### Đầu tiên, ta sẽ tải một phần nhỏ dữ liệu train để làm mẫu. (250 samples)

In [26]:
import torch
import torch.utils.data

# Read in only the first 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

# Turn the input pandas dataframe into tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

### Viết phương thức training

In [27]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            # TODO: Complete this train method to train the model provided.
            # the gradients of all optimized variables are cleared
            optimizer.zero_grad()
            # forward pass
            output = model.forward(batch_X)
            # calculate the batch loss
            loss = loss_fn(output, batch_y)
            # backpropagation
            loss.backward()
            #optimization
            optimizer.step()
            
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))

#### Kiểm tra xem phương thức ở trên có đang hoạt động hay không trên tập train vừa mới tải.

In [28]:
import torch.optim as optim
from train.model import LSTMClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

train(model, train_sample_dl, 7, optimizer, loss_fn, device)

Epoch: 1, BCELoss: 0.6918130278587341
Epoch: 2, BCELoss: 0.6828967928886414
Epoch: 3, BCELoss: 0.6747650861740112
Epoch: 4, BCELoss: 0.6651637077331543
Epoch: 5, BCELoss: 0.6523086428642273
Epoch: 6, BCELoss: 0.632752537727356
Epoch: 7, BCELoss: 0.6018641948699951


### Tạo một pytorch container chứa code train

In [29]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    py_version="py3",
                    train_instance_count=1,
                    train_instance_type='ml.m4.xlarge',
                    hyperparameters={
                        'epochs': 10,
                        'hidden_dim': 200,
                    })

Train với toàn bộ data

In [ ]:
estimator.fit({'training': input_data})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2021-11-20 14:16:37 Starting - Starting the training job...
2021-11-20 14:16:40 Starting - Launching requested ML instances......
2021-11-20 14:17:46 Starting - Preparing the instances for training.........
2021-11-20 14:19:31 Downloading - Downloading input data...
2021-11-20 14:20:03 Training - Downloading the training image...
2021-11-20 14:20:24 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-11-20 14:20:25,921 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-11-20 14:20:25,924 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-11-20 14:20:25,937 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-11-20 14:20:25,941 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-11-20 14:20:26,358 sagemaker-containers 

Epoch: 1, BCELoss: 0.6653310583562267


## Bước 5: Test model đã train

Ta sẽ triển khai model 2 lần

Lần đầu sẽ test model này bằng cách triển khai nó và lần thứ hai ta sẽ gửi dữ liệu test đến endpoint đã triển khai. Chúng ta làm điều này để đảm bảo rằng model được triển khai đang hoạt động chính xác.

## Bước 6 (lần 1): Triển khai model

Sau khi đã train model, model nhận đầu vào dạng `review_length, review [500]`, trong đó `review [500]` là một chuỗi các số nguyên `500` mô tả các từ có trong review, được mã hóa bằng cách sử dụng `word_dict`.

Tiếp theo ta sẽ triển khai model

In [1]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

NameError: name 'estimator' is not defined

## Step 7 - Sử dụng model
Lấy dữ liệu test

In [ ]:
test_X = pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1)

Sau đó test model

In [ ]:
# We split the data into chunks and send each chunk seperately, accumulating the results.
def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        predictions = np.append(predictions, predictor.predict(array))
    
    return predictions

In [ ]:
predictions = predict(test_X.values)
predictions = [round(num) for num in predictions]

Tính độ chính xác của model

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

### Thử cho một sample và test

In [ ]:
test_review = 'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'

In [ ]:
#Convert test_review into a form usable by the model and save the results in test_data
test_review_data, test_review_len = convert_and_pad_data(word_dict, review_to_words(test_review))

test_data = np.array(test_review_data)
test_data = np.insert(test_data, 0, test_review_len)

test_data = test_data[None, :]

# test_data_review_to_words = review_to_words(test_review)
# test_data = [np.array(convert_and_pad(word_dict, test_data_review_to_words)[0])]

In [ ]:
predictor.predict(test_data)

Giá trị trả về gần bằng `1`, dự đoán đây là một review thích cực

### Xóa endpoint

Xóa endpoint khi không cần sử dụng

In [80]:
estimator.delete_endpoint()

estimator.delete_endpoint() will be deprecated in SageMaker Python SDK v2. Please use the delete_endpoint() function on your predictor instead.


## Bước 6 (lần 2) - Triển khai model lên ứng dụng web

Model đang hoạt động tốt, đã đến lúc tạo một số custom inference code để có thể gửi cho model một bài đánh giá chưa được xử lý để nó xác định cảm xúc của bài đánh giá.

Code được lưu trữ trong thư mục `serve`. Trong đó có tệp `model.py` sử dụng để xây dựng model, tệp `utils.py` chứa các hàm tiền xử lý `review_to_words` và` convert_and_pad` mà đã được sử dụng trong quá trình tiền xử lý dữ liệu, và tệp `suggest.py` sẽ chứa custom inference code. Lưu ý rằng `tests.txt` sẽ cho SageMaker biết những thư viện Python nào được yêu cầu bởi custom inference code.

In [73]:
!pygmentize serve/predict.py

import argparse
import json
import os
import pickle
import sys
import sagemaker_containers
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

from model import LSTMClassifier

from utils import review_to_words, convert_and_pad

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LSTMClassifier(model_info['embedding_dim'], model_info['hidden_dim'], model_info['vocab_size'])

    # Load the store model parameters.
    model_path = os.path.join(mod

### Triển khai model

Custom inference code đã được viết, tiếp theo sẽ tạo và triển khai mô hình.

In [74]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


------------------!

### Test model

Kiểm tra xem mọi thứ có hoạt động hay không. Kiểm tra model bằng cách tải `250` đánh giá tích cực và tiêu cực đầu tiên và gửi chúng đến endpoint, sau đó quan sát kết quả.

In [75]:
import glob

def test_reviews(data_dir='../data/aclImdb', stop=250):
    
    results = []
    ground = []
    
    # We make sure to test both positive and negative reviews    
    for sentiment in ['pos', 'neg']:
        
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        
        files_read = 0
        
        print('Starting ', sentiment, ' files')
        
        # Iterate through the files and send them to the predictor
        for f in files:
            with open(f) as review:
                # First, we store the ground truth (was the review positive or negative)
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                # Read in the review and convert to 'utf-8' for transmission via HTTP
                review_input = review.read().encode('utf-8')
                # Send the review to the predictor and store the results
                # results.append(float(predictor.predict(review_input)))
                results.append(int(predictor.predict(review_input)))
                
            # Sending reviews to our endpoint one at a time takes a while so we
            # only send a small number of reviews
            files_read += 1
            if files_read == stop:
                break
            
    return ground, results

In [76]:
ground, results = test_reviews()

Starting  pos  files
Starting  neg  files


In [77]:
from sklearn.metrics import accuracy_score
accuracy_score(ground, results)

0.852

Thử kiểm tra lại `test_review` đã kiểm tra lúc trước

In [78]:
predictor.predict(test_review)

b'1'

Như vậy endpoint đang hoạt động như mong đợi (kết quả khớp với lần test trước), bây giờ ta có thể thiết lập trang web với nó.

## Bước 7 (lần 2): Sử dụng model cho web app

Chúng ta muốn tạo một ứng dụng web truy cập vào model. Cách mọi thứ được thiết lập hiện tại khiến điều đó không thể thực hiện được vì để truy cập endpoint SageMaker, trước tiên ứng dụng sẽ phải xác thực với AWS bằng IAM role bao gồm quyền truy cập vào endpoint SageMaker. Tuy nhiên, có một cách dễ dàng hơn là chỉ cần sử dụng một số dịch vụ AWS bổ sung.

<img src="Web App Diagram.svg">

Sơ đồ trên giúp ta có một cái nhìn tổng quan về cách các dịch vụ khác nhau sẽ hoạt động cùng nhau. Ở ngoài cùng bên phải là model mà ta đã đào tạo ở trên và được triển khai bằng SageMaker. Ở phía ngoài cùng bên trái là ứng dụng web của chúng tôi thu thập đánh giá phim của người dùng.

Ở giữa chúng ta tạo một Lambda function, có thể coi đây là một hàm Python đơn giản có thể được thực thi bất cứ khi nào một sự kiện cụ thể xảy ra. Chúng tôi sẽ cấp quyền cho chức năng này để gửi và nhận dữ liệu từ một endpoint của SageMaker.

Cuối cùng, phương thức sẽ sử dụng để thực thi hàm Lambda là một endpoint mới mà chúng ta sẽ tạo bằng cách sử dụng API Gateway. Endpoint này sẽ là một url lắng nghe dữ liệu được gửi đến nó. Khi nó nhận được một số dữ liệu, nó sẽ chuyển dữ liệu đó vào Lambda function và sau đó trả về bất cứ thứ gì mà hàm Lambda trả về. Về cơ bản, nó sẽ hoạt động như một giao diện cho phép ứng dụng web của chúng ta giao tiếp với Lambda function.

### Thiết lập Lambda function
Điều đầu tiên chúng ta sẽ làm là thiết lập một Lambda funtion. Hàm Lambda này sẽ được thực thi bất cứ khi nào public API có dữ liệu được gửi đến nó. Khi nó được thực thi, nó sẽ nhận dữ liệu, thực hiện bất kỳ loại xử lý nào được yêu cầu, gửi dữ liệu (đánh giá) đến endpoint mà ta đã tạo và sau đó trả về kết quả.

#### Bước A: Tạo một IAM Role cho Lambda function
Vì chúng ta muốn hàm Lambda gọi một endpoint. Để làm điều này, chúng ta sẽ xây dựng một role mà sau này chúng ta có thể cung cấp cho Lambda function.

Sử dụng bảng điều khiển AWS, điều hướng đến trang **IAM** và nhấp vào **Roles**. Sau đó, nhấp vào **Create role**. Chọn **AWS service** và chọn **Lambda** làm dịch vụ sẽ sử dụng, sau đó nhấp vào **Next: Permission**.

Trong search box, nhập `sagemaker` và chọn check box bên cạnh **AmazonSageMakerFullAccess** sau đó click **Next: Tags**. Sau đó, click vào **Next: Review**.

Cuối cùng, đặt tên cho role này. Ví dụ: `LambdaSageMakerRole`. Sau đó, nhấp vào **Create role**.

#### Bước B: Tạo một Lambda function
Sử dụng bảng điều khiển AWS, điều hướng đến trang AWS Lambda và nhấp vào **Create a function**. Khi đến trang tiếp theo, hãy đảm bảo rằng **Author from scratch** được chọn. Tiếp theo thì đặt tên cho hàm Lambda, ví dụ như `feel_analysis_func`. Đảm bảo rằng **Python 3.6** runtime được chọn và sau đó chọn role đã tạo trong phần trước. Sau đó, nhấp vào **Create Function**.

Trên trang tiếp theo, ta sẽ thấy một số thông tin về hàm Lambda mà ta vừa tạo. Nếu cuộn xuống ta sẽ thấy một trình soạn thảo, trong đó ta sẽ viết code sẽ được thực thi khi chức năng Lambda được kích hoạt. Chúng ta sẽ dùng đoạn mã sau:

```python
# We need to use the low-level library to interact with SageMaker since the SageMaker API
# is not available natively through Lambda.
import boto3

def lambda_handler(event, context):

    # The SageMaker runtime is what allows us to invoke the endpoint that we've created.
    runtime = boto3.Session().client('sagemaker-runtime')

    # Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given
    response = runtime.invoke_endpoint(EndpointName = '**ENDPOINT NAME HERE**',    # The name of the endpoint we created
                                       ContentType = 'text/plain',                 # The data format that is expected
                                       Body = event['body'])                       # The actual review

    # The response is an HTTP response whose body contains the result of our inference
    result = response['Body'].read().decode('utf-8')

    return {
        'statusCode' : 200,
        'headers' : { 'Content-Type' : 'text/plain', 'Access-Control-Allow-Origin' : '*' },
        'body' : result
    }
```


Khi đã sao chép và dán đoạn mã trên vào trình chỉnh sửa code Lambda, hãy thay thế phần `**ENDPOINT NAME HERE**' bằng tên của endpoint mà chúng ta đã triển khai trước đó. Có thể xác định tên của endpoint bằng cách chạy code bên dưới.


In [79]:
predictor.endpoint

'sagemaker-pytorch-2021-11-20-13-16-51-967'

Khi đã thêm tên endpoint vào hàm Lambda, hãy nhấn vào **Save**. Lambda function hiện đã hoạt động. Tiếp theo, chúng ta cần tạo một cách để ứng dụng web của chúng ta thực thi Lambda function.

### Thiết lập API Gateway
Bây giờ Lambda function của chúng ta đã được thiết lập, tiếp theo ta sẽ tạo một API mới bằng cách sử dụng API Gateway sẽ kích hoạt hàm Lambda mà chúng ta vừa tạo.

Sử dụng bảng điều khiển AWS, điều hướng đến **Amazon API Gateway** và sau đó nhấp vào **Get started**.

Trên trang tiếp theo, chọn **New API** và đặt tên cho api mới, ví dụ: `sentiment_analysis_api`. Sau đó, nhấp vào **Create API**.

Bây giờ chúng ta đã tạo một API, tuy nhiên nó hiện nó không có tác dụng gì. Những gì ta muốn nó làm là kích hoạt Lambda function mà chúng ta đã tạo trước đó.

Chọn dropdown menu **Action** và nhấp vào **Create Method**. Một phương thức trống mới sẽ được tạo, hãy chọn dropdown menu của nó và chọn **POST**, sau đó click vào dấu check bên cạnh nó.

Đối với integration point, hãy đảm bảo rằng **Lambda Function** được chọn và click vào **Use Lambda Proxy integration**. Tùy chọn này đảm bảo rằng dữ liệu được gửi đến API sau đó sẽ được gửi trực tiếp đến hàm Lambda mà không cần xử lý. Điều đó cũng có nghĩa là giá trị trả về phải là một response object thích hợp vì nó cũng sẽ không được xử lý bởi API Gateway.

Nhập tên của Lambda function mà ta đã tạo trước đó vào text box **Lambda Function** và sau đó click vào **Save**. Click vào **OK** trong pop-up box xuất hiện, cấp quyền cho API Gateway để gọi Lambda function mà ta đã tạo.

Bước cuối cùng trong việc tạo API Gateway là chọn dropdown menu **Actions** và nhấp vào **Deploy API**. Sau đó đặt tên cho nó ví dụ như `prod`.

Bây giờ ta đã thiết lập thành công một public API để truy cập vào model của mình. Copy URL được cung cấp để gọi public API mới tạo vì điều này sẽ cần thiết trong bước tiếp theo. Ta có thể tìm thấy URL này ở đầu trang, được đánh dấu bằng màu xanh lam bên cạnh dòng chữ **Invoke URL**.

## Bước cuối: Triển khai ứng dụng web

Bây giờ ta đã có một public API, chúng ta có thể bắt đầu sử dụng nó trong một ứng dụng web.

Trong thư mục `website` có một tệp có tên là `index.html`. Tải tệp xuống máy tính và mở tệp đó trong trình soạn thảo văn bản nào đó. Tìm dòng chữ ****REPLACE WITH PUBLIC API URL****. Thay thế chuỗi này bằng url mà ta đã có được.

Now, if you open `index.html` on your local computer, your browser will behave as a local web server and you can use the provided site to interact with your SageMaker model.

Bây giờ, nếu mở `index.html` trên máy tính của chúng ta, trình duyệt sẽ hoạt động như một máy chủ web cục bộ và ta có thể sử dụng trang web để tương tác với SageMaker Model của mình.

> ** Lưu ý quan trọng ** Để ứng dụng web giao tiếp với endpoint, endpoint phải thực sự được triển khai và chạy. Điều này có nghĩa là ta đang trả tiền cho nó. Đảm bảo rằng endpoint đang chạy khi ta muốn sử dụng ứng dụng web nhưng ta nên tắt nó khi không cần đến, nếu không sẽ nhận được một hóa đơn rất lớn.

Bây giờ ứng dụng web đang hoạt động và đây là kết quả:

**Ảnh chụp màn hình:**



**Kết quả:**

### Xóa endpoint
Tắt endpoint khi không còn sử dụng nó nữa. Nếu không sẽ tốn rất nhiều tiền.

In [81]:
predictor.delete_endpoint()